In [1]:
import pandas as pd
import numpy as np

### Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

##### Making dataframe with Postal Codes of Toronto

In [3]:
#To proceed request to Wikipedia we need som libraries
from urllib.request import urlopen, urlretrieve
from bs4 import BeautifulSoup

In [5]:
#sending request to Wikipedia and getting result to BS
source = urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read().decode('utf8')
soup = BeautifulSoup(source,'lxml')

#let's find our table by class (you can find class by right-click => inspect on the table) and get all the rows in there
table = soup.find('table', attrs={'class':'wikitable sortable'})
table_rows = table.find_all('tr')

#rows proceed
rows_content = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.rstrip() for tr in td]
    rows_content.append(row)

#making dataframe
table_raw = pd.DataFrame(rows_content, columns=["Postal Code", "Borough", "Neighbourhood"])
table = table_raw[table_raw['Borough'] != 'Not assigned']
table = table[1:]
table.shape

(103, 3)